In this notebook, I calculated cosine similarity between self-determination embedding and transcript embeddings. This time, for transcript embeddings, I used the embeddings representing a whole transcript. After that, I ranked the transcripts by the cosine siamilarity.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
pip install -U sentence-transformers

     |████████████████████████████████| 79 kB 5.7 MB/s 
     |████████████████████████████████| 3.8 MB 28.0 MB/s 
     |████████████████████████████████| 1.2 MB 52.8 MB/s 
     |████████████████████████████████| 67 kB 5.2 MB/s 
     |████████████████████████████████| 895 kB 47.1 MB/s 
     |████████████████████████████████| 6.5 MB 34.2 MB/s 
     |████████████████████████████████| 596 kB 42.3 MB/s 
  Created wheel for sentence-transformers: filename=sentence_transformers-2.2.0-py3-none-any.whl size=120747 sha256=b657f259b562140bc2dca4d438fc5b75e3e22431e3a60284e429f7bc1a8597ae
  Stored in directory: /root/.cache/pip/wheels/83/c0/df/b6873ab7aac3f2465aa9144b6b4c41c4391cfecc027c8b07e7
Successfully built sentence-transformers
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [ ]:
# data access
import pandas as pd

import numpy as np
from numpy import dot
from numpy.linalg import norm

# Transformers Library
from sentence_transformers import SentenceTransformer

In [ ]:
df = pd.read_csv('/content/drive/MyDrive/Capstone Design/data/processed_data/embedding_df.csv')
mega_df = pd.read_csv('/content/drive/MyDrive/Capstone Design/data/processed_data/processed_data.csv')

In [ ]:
df = df.drop('Unnamed: 0', axis = 1)

In [ ]:
df.head()

,ele1,ele2,ele3,ele4,ele5,ele6,ele7,ele8,ele9,ele10,...,ele376,ele377,ele378,ele379,ele380,ele381,ele382,ele383,ele384,category
0,0.411153,-0.079363,0.328998,-0.126358,-0.088267,-0.055317,-0.114667,-0.075146,0.012047,-0.310470,...,-0.054759,0.079325,-0.199157,-0.257087,-0.043128,-0.025510,0.126650,-0.196273,0.209506,speaker_transcript
1,0.090411,-0.071218,0.293163,0.146878,-0.115421,-0.135240,0.285174,0.127226,-0.025668,-0.161996,...,-0.030386,0.233547,-0.018198,-0.126653,-0.022223,0.264613,0.227376,-0.436245,0.081586,speaker_transcript
2,0.131044,-0.208772,0.213030,-0.072059,-0.294220,0.114275,-0.041299,0.026787,-0.315146,-0.224988,...,-0.076660,0.242875,0.046984,-0.417606,-0.059260,-0.100854,0.282876,-0.221022,-0.004070,speaker_transcript
3,0.192777,-0.293854,0.007482,-0.170282,-0.576680,0.001538,0.121933,0.004217,-0.143814,-0.197574,...,0.239390,0.076596,-0.070537,-0.353680,0.016037,0.174575,0.175027,-0.041052,-0.279887,speaker_transcript
4,0.426892,-0.137657,0.210333,0.033745,-0.387424,-0.302002,-0.013708,0.091537,-0.296233,-0.203553,...,0.095008,0.109985,-0.209231,-0.280066,-0.186285,-0.006425,0.013543,-0.496376,0.248774,speaker_transcript


In [ ]:
df.shape

(596, 385)

In [ ]:
sit_emb_df = df[df['category'].isin(['choice_make_sit', 'decision_make_sit', 'prob_sov_sit', 'goal_set_sit', 'goal_att_sit', 'self_advocacy_sit', 'self_mng_sit', 'self_aware_sit'])]

In [ ]:
sit_emb_df

,ele1,ele2,ele3,ele4,ele5,ele6,ele7,ele8,ele9,ele10,...,ele376,ele377,ele378,ele379,ele380,ele381,ele382,ele383,ele384,category
588,0.055173,0.213760,0.122395,-0.141248,0.072153,-0.064798,0.310212,-0.100451,-0.009148,-0.093294,...,0.346108,0.028083,-0.107546,-0.419796,0.220396,-0.080600,0.057738,0.133060,0.331578,choice_make_sit
589,-0.020836,0.226021,-0.298644,-0.260912,0.060680,0.140059,0.742423,-0.013295,0.142886,0.210192,...,0.235111,0.315617,-0.280026,0.005863,0.294901,-0.350100,0.333456,-0.045025,0.193500,decision_make_sit
590,-0.259151,0.206800,0.243713,0.070021,0.226014,-0.019762,0.443001,0.132121,-0.238051,0.225958,...,0.178570,0.290097,-0.179001,-0.294276,0.062933,-0.075940,0.618843,-0.112328,0.357486,prob_sov_sit
591,-0.423885,0.190180,-0.058502,-0.389208,-0.187806,0.214891,0.310606,-0.209747,-0.089531,-0.014026,...,0.506565,0.361855,-0.190027,-0.306028,0.339284,0.187701,0.145460,-0.144122,0.173004,goal_set_sit
592,-0.017430,0.345679,-0.085979,-0.389166,0.183177,-0.213084,0.736760,0.112504,-0.153969,0.205149,...,0.509390,0.151046,0.131825,-0.033526,0.338645,-0.130954,0.385607,-0.243900,0.262245,goal_att_sit
593,-0.010024,0.243337,-0.293931,-0.176108,-0.102239,0.108640,0.273495,-0.433400,0.017285,0.176631,...,0.227962,0.238206,0.041550,-0.241106,0.243534,0.101449,0.223140,0.114820,0.204630,self_advocacy_sit
594,-0.213198,0.166392,0.151563,-0.302374,-0.275009,0.329999,0.583866,-0.064867,-0.404409,0.296618,...,0.287275,0.354076,-0.024666,-0.306770,0.131460,0.210675,0.202623,0.344589,0.113316,self_mng_sit
595,0.054310,0.093643,0.176992,-0.370356,0.120260,-0.342370,1.020324,0.156827,-0.050656,-0.091770,...,0.492050,-0.016455,0.232713,-0.446013,0.133615,0.483060,0.365996,-0.217936,0.321546,self_aware_sit


In [ ]:
df['category'].unique()

array(['speaker_transcript', 'no_speaker_transcript', 'word',
       'definition', 'choice_make_sit', 'decision_make_sit',
       'prob_sov_sit', 'goal_set_sit', 'goal_att_sit',
       'self_advocacy_sit', 'self_mng_sit', 'self_aware_sit'],
      dtype=object)

In [ ]:
sit_avg_emb = []
for i in range(sit_emb_df.shape[1]-1):
  result = sit_emb_df['ele' + str(i+1)].mean()
  sit_avg_emb.append(result)

In [ ]:
len(sit_avg_emb)

384

In [ ]:
transcript_df = df[df['category'].isin(['speaker_transcript', 'speaker_transcript', 'no_speaker_transcript'])]

In [ ]:
transcript_df.shape

(586, 385)

In [ ]:
transcript_df.head()

,ele1,ele2,ele3,ele4,ele5,ele6,ele7,ele8,ele9,ele10,...,ele376,ele377,ele378,ele379,ele380,ele381,ele382,ele383,ele384,category
0,0.411153,-0.079363,0.328998,-0.126358,-0.088267,-0.055317,-0.114667,-0.075146,0.012047,-0.310470,...,-0.054759,0.079325,-0.199157,-0.257087,-0.043128,-0.025510,0.126650,-0.196273,0.209506,speaker_transcript
1,0.090411,-0.071218,0.293163,0.146878,-0.115421,-0.135240,0.285174,0.127226,-0.025668,-0.161996,...,-0.030386,0.233547,-0.018198,-0.126653,-0.022223,0.264613,0.227376,-0.436245,0.081586,speaker_transcript
2,0.131044,-0.208772,0.213030,-0.072059,-0.294220,0.114275,-0.041299,0.026787,-0.315146,-0.224988,...,-0.076660,0.242875,0.046984,-0.417606,-0.059260,-0.100854,0.282876,-0.221022,-0.004070,speaker_transcript
3,0.192777,-0.293854,0.007482,-0.170282,-0.576680,0.001538,0.121933,0.004217,-0.143814,-0.197574,...,0.239390,0.076596,-0.070537,-0.353680,0.016037,0.174575,0.175027,-0.041052,-0.279887,speaker_transcript
4,0.426892,-0.137657,0.210333,0.033745,-0.387424,-0.302002,-0.013708,0.091537,-0.296233,-0.203553,...,0.095008,0.109985,-0.209231,-0.280066,-0.186285,-0.006425,0.013543,-0.496376,0.248774,speaker_transcript


## 1. Cosine Similarity

In [ ]:
temp = transcript_df.drop('category', axis = 1)

In [ ]:
temp

,ele1,ele2,ele3,ele4,ele5,ele6,ele7,ele8,ele9,ele10,...,ele375,ele376,ele377,ele378,ele379,ele380,ele381,ele382,ele383,ele384
0,0.411153,-0.079363,0.328998,-0.126358,-0.088267,-0.055317,-0.114667,-0.075146,0.012047,-0.310470,...,0.584021,-0.054759,0.079325,-0.199157,-0.257087,-0.043128,-0.025510,0.126650,-0.196273,0.209506
1,0.090411,-0.071218,0.293163,0.146878,-0.115421,-0.135240,0.285174,0.127226,-0.025668,-0.161996,...,0.317512,-0.030386,0.233547,-0.018198,-0.126653,-0.022223,0.264613,0.227376,-0.436245,0.081586
2,0.131044,-0.208772,0.213030,-0.072059,-0.294220,0.114275,-0.041299,0.026787,-0.315146,-0.224988,...,0.359164,-0.076660,0.242875,0.046984,-0.417606,-0.059260,-0.100854,0.282876,-0.221022,-0.004070
3,0.192777,-0.293854,0.007482,-0.170282,-0.576680,0.001538,0.121933,0.004217,-0.143814,-0.197574,...,0.352274,0.239390,0.076596,-0.070537,-0.353680,0.016037,0.174575,0.175027,-0.041052,-0.279887
4,0.426892,-0.137657,0.210333,0.033745,-0.387424,-0.302002,-0.013708,0.091537,-0.296233,-0.203553,...,0.400299,0.095008,0.109985,-0.209231,-0.280066,-0.186285,-0.006425,0.013543,-0.496376,0.248774
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
581,0.276333,-0.133476,0.530620,0.104498,-0.300166,-0.183760,-0.161849,0.338993,-0.112626,-0.076315,...,0.252306,-0.131236,-0.061268,-0.042192,-0.462636,-0.007985,0.473416,0.037567,-0.305676,0.191065
582,0.060837,-0.050574,0.121308,0.061480,-0.434407,-0.291122,-0.172725,0.176536,-0.340843,-0.416462,...,0.438940,-0.194455,-0.073258,-0.182421,-0.432297,-0.188575,0.217918,-0.014680,-0.364656,-0.048277
583,-0.011974,-0.445796,0.050308,0.051568,-0.280475,0.034414,-0.111923,0.220812,-0.224556,0.018741,...,0.304454,-0.324890,-0.107352,-0.296290,-0.240207,0.209208,-0.561485,-0.123841,0.115077,-0.061878
584,0.354133,-0.434064,0.154213,0.142982,-0.318219,-0.066567,-0.188593,0.061227,-0.041086,0.040959,...,0.203482,0.170708,0.059035,0.124811,-0.704490,0.274732,0.283143,-0.008834,-0.178593,-0.076478


In [ ]:
filename = mega_df['file_name'].unique().tolist()

In [ ]:
filenames = filename + filename

In [ ]:
temp['filename'] = filenames

In [ ]:
temp.head()

,ele1,ele2,ele3,ele4,ele5,ele6,ele7,ele8,ele9,ele10,...,ele376,ele377,ele378,ele379,ele380,ele381,ele382,ele383,ele384,filename
0,0.411153,-0.079363,0.328998,-0.126358,-0.088267,-0.055317,-0.114667,-0.075146,0.012047,-0.310470,...,-0.054759,0.079325,-0.199157,-0.257087,-0.043128,-0.025510,0.126650,-0.196273,0.209506,2039_parent_11.04.2021.docx
1,0.090411,-0.071218,0.293163,0.146878,-0.115421,-0.135240,0.285174,0.127226,-0.025668,-0.161996,...,-0.030386,0.233547,-0.018198,-0.126653,-0.022223,0.264613,0.227376,-0.436245,0.081586,2033_VocIndex-and-SA_07.09.2020.docx
2,0.131044,-0.208772,0.213030,-0.072059,-0.294220,0.114275,-0.041299,0.026787,-0.315146,-0.224988,...,-0.076660,0.242875,0.046984,-0.417606,-0.059260,-0.100854,0.282876,-0.221022,-0.004070,2035_GAS-and-VocIndex_06.18.2020.docx
3,0.192777,-0.293854,0.007482,-0.170282,-0.576680,0.001538,0.121933,0.004217,-0.143814,-0.197574,...,0.239390,0.076596,-0.070537,-0.353680,0.016037,0.174575,0.175027,-0.041052,-0.279887,2040_VocIndex_06.19.2020.docx
4,0.426892,-0.137657,0.210333,0.033745,-0.387424,-0.302002,-0.013708,0.091537,-0.296233,-0.203553,...,0.095008,0.109985,-0.209231,-0.280066,-0.186285,-0.006425,0.013543,-0.496376,0.248774,2034_GAS_Voc_SA_05.18.2020.docx


In [ ]:
temp_cal = temp.copy()
temp_cal = temp_cal.drop('filename', axis = 1)

In [ ]:
cosine_similarity = []
for i in range(temp_cal.shape[0]):
  transcript_emb = temp_cal.iloc[i].tolist()
  result = dot(sit_avg_emb, transcript_emb)/(norm(sit_avg_emb)*norm(transcript_emb))
  cosine_similarity.append(result)

In [ ]:
len(cosine_similarity)

586

In [ ]:
data = {'filename': filenames, 'cosine_similarity': cosine_similarity}
cos_sim_df = pd.DataFrame(data)

In [ ]:
cos_sim_df

,filename,cosine_similarity
0,2039_parent_11.04.2021.docx,0.065311
1,2033_VocIndex-and-SA_07.09.2020.docx,0.017045
2,2035_GAS-and-VocIndex_06.18.2020.docx,0.150822
3,2040_VocIndex_06.19.2020.docx,0.191691
4,2034_GAS_Voc_SA_05.18.2020.docx,-0.001988
...,...,...
581,1060_voc index_06.17.2021.docx,0.102030
582,1059_VocParent_VocYouth_07.23.2020.docx,0.048255
583,3008_T3-12-mo-Parent-and-Youth_11.13.2021.docx,0.239910
584,3009_ Parent 12-Month Follow Up Interview_10.2...,0.319916


In [ ]:
cos_sim_df['category'] = transcript_df['category'].tolist()

speaker_df = cos_sim_df[cos_sim_df['category'] == "speaker_transcript"]
no_speaker_df = cos_sim_df[cos_sim_df['category'] != "speaker_transcript"]

In [ ]:
speaker_sit_avg_similarity = speaker_df.sort_values(by = 'cosine_similarity', ascending = False)

In [ ]:
speaker_sit_avg_similarity = speaker_sit_avg_similarity.reset_index(drop = True)

In [ ]:
speaker_sit_avg_similarity.head()

,filename,cosine_similarity,category
0,3009_ Parent 12-Month Follow Up Interview_10.2...,0.304603,speaker_transcript
1,1027_Vineland_VocIndex_02.06.2020.docx,0.298736,speaker_transcript
2,3014_T2-Service-Access-and-VocIndex_05.18.2021...,0.286378,speaker_transcript
3,1061_Voc_Service_07.23.2020.docx,0.279917,speaker_transcript
4,1020_voc index youth_08.20.2021.docx,0.279270,speaker_transcript


In [ ]:
speaker_sit_avg_similarity.to_csv("speaker_sit_avg_similarity.csv")

In [ ]:
no_speaker_sit_avg_similarity = no_speaker_df.sort_values(by = 'cosine_similarity', ascending = False)

In [ ]:
no_speaker_sit_avg_similarity = no_speaker_sit_avg_similarity.reset_index(drop = True)

In [ ]:
no_speaker_sit_avg_similarity

,filename,cosine_similarity,category
0,3009_ Parent 12-Month Follow Up Interview_10.2...,0.319916,no_speaker_transcript
1,3014_T2-Service-Access-and-VocIndex_05.18.2021...,0.290562,no_speaker_transcript
2,1020_voc index youth_08.20.2021.docx,0.290163,no_speaker_transcript
3,1027_Vineland_VocIndex_02.06.2020.docx,0.269671,no_speaker_transcript
4,1007_VocYouth_01.09.2020.docx,0.250985,no_speaker_transcript
...,...,...,...
288,2053_parent_11.01.2021.docx,-0.052376,no_speaker_transcript
289,1006_voc index_01.06.2021.docx,-0.055420,no_speaker_transcript
290,2061_VocIndex_07.07.2020.docx,-0.070913,no_speaker_transcript
291,1045_voc index_youth_12.20.2021.docx,-0.090735,no_speaker_transcript


In [ ]:
no_speaker_sit_avg_similarity.to_csv("no_speaker_sit_avg_similarity.csv")

# Cosine similarity: word - transcript 

In [ ]:
model = SentenceTransformer('paraphrase-MiniLM-L6-v2')

word_embedding = model.encode('self-determination')

Downloading:   0%|          | 0.00/690 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/3.69k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/629 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/122 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/229 [00:00<?, ?B/s]

...

Downloading:   0%|          | 0.00/112 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/466k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/314 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [ ]:
word_emb = word_embedding.tolist()

In [ ]:
cosine_similarity_word = []
for i in range(temp_cal.shape[0]):
  transcript_emb = temp_cal.iloc[i].tolist()
  result = dot(word_emb, transcript_emb)/(norm(word_emb)*norm(transcript_emb))
  cosine_similarity_word.append(result)

In [ ]:
data = {'filename': filenames, 'cosine_similarity': cosine_similarity_word}
cos_sim_word_df = pd.DataFrame(data)

In [ ]:
cos_sim_word_df

,filename,cosine_similarity
0,2039_parent_11.04.2021.docx,-0.000908
1,2033_VocIndex-and-SA_07.09.2020.docx,-0.027807
2,2035_GAS-and-VocIndex_06.18.2020.docx,0.065966
3,2040_VocIndex_06.19.2020.docx,0.073640
4,2034_GAS_Voc_SA_05.18.2020.docx,0.024039
...,...,...
581,1060_voc index_06.17.2021.docx,0.068396
582,1059_VocParent_VocYouth_07.23.2020.docx,-0.008164
583,3008_T3-12-mo-Parent-and-Youth_11.13.2021.docx,0.083522
584,3009_ Parent 12-Month Follow Up Interview_10.2...,0.191398


In [ ]:
cos_sim_word_df['category'] = transcript_df['category'].tolist()

speaker_word_df = cos_sim_word_df[cos_sim_word_df['category'] == "speaker_transcript"]
no_speaker_word_df = cos_sim_word_df[cos_sim_word_df['category'] != "speaker_transcript"]

In [ ]:
cos_sim_word_df

,filename,cosine_similarity,category
0,2039_parent_11.04.2021.docx,-0.000908,speaker_transcript
1,2033_VocIndex-and-SA_07.09.2020.docx,-0.027807,speaker_transcript
2,2035_GAS-and-VocIndex_06.18.2020.docx,0.065966,speaker_transcript
3,2040_VocIndex_06.19.2020.docx,0.073640,speaker_transcript
4,2034_GAS_Voc_SA_05.18.2020.docx,0.024039,speaker_transcript
...,...,...,...
581,1060_voc index_06.17.2021.docx,0.068396,no_speaker_transcript
582,1059_VocParent_VocYouth_07.23.2020.docx,-0.008164,no_speaker_transcript
583,3008_T3-12-mo-Parent-and-Youth_11.13.2021.docx,0.083522,no_speaker_transcript
584,3009_ Parent 12-Month Follow Up Interview_10.2...,0.191398,no_speaker_transcript


In [ ]:
speaker_word_similarity = speaker_word_df.sort_values(by = 'cosine_similarity', ascending = False)

In [ ]:
speaker_word_similarity = speaker_word_similarity.reset_index(drop = True)

In [ ]:
speaker_word_similarity.to_csv("speaker_word_similarity.csv")

In [ ]:
no_speaker_word_similarity = no_speaker_word_df.sort_values(by = 'cosine_similarity', ascending = False)

In [ ]:
no_speaker_word_similarity = no_speaker_word_similarity.reset_index(drop = True)

In [ ]:
no_speaker_word_similarity.to_csv("no_speaker_word_similarity.csv")

## Cosine similarity between transcript and the definition

In [ ]:
definition = 'dispositional characteristic manifested as acting as the causal agent in one’s life. Self-determined people act in service to freely chosen goals. Self-determined actions function to enable a person to be the causal agent is his or her life.'

In [ ]:
def_embedding = model.encode(definition)

In [ ]:
def_embedding = def_embedding.tolist()

In [ ]:
cosine_similarity_def = []
for i in range(temp_cal.shape[0]):
  transcript_emb = temp_cal.iloc[i].tolist()
  result = dot(def_embedding, transcript_emb)/(norm(def_embedding)*norm(transcript_emb))
  cosine_similarity_def.append(result)

In [ ]:
data = {'filename': filenames, 'cosine_similarity': cosine_similarity_def}
cos_sim_def_df = pd.DataFrame(data)

In [ ]:
cos_sim_def_df

,filename,cosine_similarity
0,2039_parent_11.04.2021.docx,-0.059524
1,2033_VocIndex-and-SA_07.09.2020.docx,-0.116810
2,2035_GAS-and-VocIndex_06.18.2020.docx,-0.007755
3,2040_VocIndex_06.19.2020.docx,0.041562
4,2034_GAS_Voc_SA_05.18.2020.docx,-0.037652
...,...,...
581,1060_voc index_06.17.2021.docx,-0.071958
582,1059_VocParent_VocYouth_07.23.2020.docx,-0.078796
583,3008_T3-12-mo-Parent-and-Youth_11.13.2021.docx,0.024332
584,3009_ Parent 12-Month Follow Up Interview_10.2...,0.158969


In [ ]:
cos_sim_def_df['category'] = transcript_df['category'].tolist()

In [ ]:
cos_sim_def_df

,filename,cosine_similarity,category
0,2039_parent_11.04.2021.docx,-0.059524,speaker_transcript
1,2033_VocIndex-and-SA_07.09.2020.docx,-0.116810,speaker_transcript
2,2035_GAS-and-VocIndex_06.18.2020.docx,-0.007755,speaker_transcript
3,2040_VocIndex_06.19.2020.docx,0.041562,speaker_transcript
4,2034_GAS_Voc_SA_05.18.2020.docx,-0.037652,speaker_transcript
...,...,...,...
581,1060_voc index_06.17.2021.docx,-0.071958,no_speaker_transcript
582,1059_VocParent_VocYouth_07.23.2020.docx,-0.078796,no_speaker_transcript
583,3008_T3-12-mo-Parent-and-Youth_11.13.2021.docx,0.024332,no_speaker_transcript
584,3009_ Parent 12-Month Follow Up Interview_10.2...,0.158969,no_speaker_transcript


In [ ]:
speaker_def_df = cos_sim_def_df[cos_sim_def_df['category'] == "speaker_transcript"]
no_speaker_def_df = cos_sim_def_df[cos_sim_def_df['category'] != "speaker_transcript"]

In [ ]:
speaker_def_similarity = speaker_def_df.sort_values(by = 'cosine_similarity', ascending = False)

In [ ]:
speaker_def_similarity = speaker_def_similarity.reset_index(drop = True)

In [ ]:
speaker_def_similarity.to_csv("speaker_def_similarity.csv")

In [ ]:
no_speaker_def_similarity = no_speaker_def_df.sort_values(by = 'cosine_similarity', ascending = False)

In [ ]:
no_speaker_def_similarity = no_speaker_def_similarity.reset_index(drop = True)

In [ ]:
no_speaker_def_similarity.to_csv("no_speaker_def_similarity.csv")